In [1]:
import seaborn as sns
import pandas as pd
import numpy as np
import warnings
from sklearn.linear_model import LogisticRegression as LR
from sklearn.model_selection import cross_validate as cv
from lightgbm import LGBMClassifier as LG
from sklearn.metrics import classification_report as cr
import plotly.offline as pyo
import plotly.graph_objs as go

# Set notebook mode to work in offline
pyo.init_notebook_mode()
import plotly.io as pio
pio.renderers.default = 'iframe'
warnings.filterwarnings("ignore")



In [6]:
from holisticai.datasets import load_dataset
from holisticai.bias.plots import bias_metrics_report as bmr
from holisticai.bias.metrics import classification_bias_metrics
from holisticai.efficacy.metrics import classification_efficacy_metrics

from holisticai.bias.mitigation import (
    FairletClusteringPreprocessing,
    CorrelationRemover,
    DisparateImpactRemover,
    LearningFairRepresentation,
    Reweighing,
    AdversarialDebiasing,
    PrejudiceRemover,
    MetaFairClassifier,
    GridSearchReduction,
    ExponentiatedGradientReduction,
    CalibratedEqualizedOdds,
    EqualizedOdds,
    RejectOptionClassification,
    LPDebiaserBinary,
)

from holisticai.bias.plots import bias_metrics_report

from holisticai.security.metrics import (
    data_minimization_score,
    classification_privacy_metrics,
)
from holisticai.security.commons import DataMinimizer

from holisticai.robustness.metrics import (
    accuracy_degradation_profile,
    accuracy_degradation_factor,
    
)
from holisticai.robustness.attackers import *

from holisticai.explainability.metrics import (
    classification_global_feature_importance_explainability_metrics,
    classification_explainability_metrics,
)
from holisticai.inspection import (
    compute_permutation_importance,
    compute_partial_dependence,
    compute_conditional_permutation_importance,
)
from holisticai.utils import BinaryClassificationProxy
from holisticai.utils.surrogate_models import create_surrogate_model


In [7]:
def baseline(dataset_name='bank_marketing', random_state=42):
    
    if dataset_name == 'bank_marketing':
        prot = 'marital'
    elif dataset_name=='mw_small':
        
        prot = 'race'
    else:
        
        prot = 'sex'
        
    #Loading in data
    dataset = load_dataset(dataset_name, protected_attribute=prot, preprocessed=True)
    split_dataset = dataset.train_test_split(test_size=0.2, random_state=random_state)
    train = split_dataset['train']
    test = split_dataset['test']
    
    #Create Model and proxy
    model = LG(verbose=-1).fit(train['X'], train['y'])

    proxy = BinaryClassificationProxy(predict=model.predict, predict_proba=model.predict_proba)
    pred = model.predict(test['X'])
    
    
    #Create bias and performance metrics
    bias = classification_bias_metrics(test['group_a'], test['group_b'], pred, test['y'])
    bias_loss = abs(bias['Reference'] - bias['Value']).mean()
    
    #Create efficacy metrics, focusing on balanced acc and f1-score
    effic = classification_efficacy_metrics(pred, test['y'])
    effic = effic.iloc[[1,4]]
    effic_loss = abs(effic['Reference'] - effic['Value']).mean()
    display(bias)
    display(effic)

    # Feature Importance
    importance = compute_permutation_importance(proxy, X=train['X'], y=train['y'], n_repeats=1)
    top_importance = importance.top_alpha()
    top_feature_names = top_importance.feature_names

    # Conditional Feature Importance
    conditional_impotance = compute_conditional_permutation_importance(proxy, X=test['X'], y=test['y'])

    # Partial Dependence
    X_sample = test['X'].sample(1000)
    partial_dependence = compute_partial_dependence(features=top_feature_names, proxy=proxy, X=X_sample)
    
    #Compute and display 2 diff XAI metrics
    feat_imp = classification_global_feature_importance_explainability_metrics(partial_dependence, importance, conditional_impotance)
    feat_imp_2 = classification_explainability_metrics(importance, partial_dependence, conditional_impotance)
    feat_imp_2.columns = feat_imp.columns
    
    final_feat = pd.concat([feat_imp, feat_imp_2], axis=0).drop_duplicates()
    feat_loss = abs(final_feat['Reference'] - final_feat['Value']).mean()
    display(final_feat)
    
    total = bias_loss + effic_loss + feat_loss
    
    loss = [bias_loss, effic_loss, feat_loss]
    
    print("Loss per metric:")
    print("Bias Loss:", bias_loss)
    print("Performance Loss:", effic_loss)
    print("Explainability Loss:", feat_loss)
    print("Total Loss:", total)

In [4]:
def split(dataset_name='bank_marketing', random_state=42):
    
    pro_dic={"bank_marketing":"marital",
            "mw_small":"race",
            "compas_two_year_recid":"sex"}
    
    prot = pro_dic[dataset_name]
    
    dataset = load_dataset(dataset_name, protected_attribute=prot, preprocessed=True)
    split_dataset = dataset.train_test_split(test_size=0.2, random_state=random_state)
    
    train = split_dataset['train']
    test = split_dataset['test']
    hidden_dataset = test.train_test_split(test_size=0.2, random_state=random_state)
    test = hidden_dataset['train']
    hidden = hidden_dataset['test']
    
    return train, test, hidden

In [5]:
# Define the objective function for Optuna

pre_mitigators = [
                   [None, None],
                   ['CorrelationRemover',CorrelationRemover()],
                   ['DisparateImpactRemover',DisparateImpactRemover()],
                   ['LearningFairRepresentation',LearningFairRepresentation()],
                   ['Reweighing',Reweighing()]
    
                    ]

in_mitigators = [
                    [None, None],
                    ["MetaFairClassifier", MetaFairClassifier(verbose=2)],
                    ['GridSearchReduction', GridSearchReduction()],
                    ['ExponentiatedGradientReduction', ExponentiatedGradientReduction()],
                ]

In [6]:
def process_and_evaluate(method_name, proc, train, test):
    print(method_name)
    
    # Fit and transform the data
    if method_name in ["CorrelationRemover", "DisparateImpactRemover"]:
        train_mod = proc.fit_transform(train["X"], train["group_a"], train["group_b"])
        test_mod = proc.transform(test["X"], test["group_a"], test["group_b"])
        
        train_mod = np.array(train_mod)
        test_mod = np.array(test_mod)
        
    elif method_name in ["LearningFairRepresentation", "Reweighing"]:
        train_mod = proc.fit_transform(train["X"], train["y"], train["group_a"], train["group_b"])
        test_mod = proc.transform(test["X"], test["group_a"], test["group_b"])
    elif method_name == None:
        train_mod = train['X'].copy()
        test_mod = test['X'].copy()
    
    if method_name == 'Reweighing':
        sample_weight = proc.sample_weight
        fit_params = {"sample_weight": sample_weight}
    else:
        fit_params={}
        sample_weight = np.ones(shape=(len(train['X'])))
        
    train_mod=pd.DataFrame(train_mod, columns=train['X'].columns)#.values
    test_mod =pd.DataFrame(test_mod, columns=test['X'].columns)#.values
    # Train the model
    fit_params = {"sample_weight": sample_weight} if sample_weight is not None else {}
    model = LG(verbose=-1).fit(train_mod, train["y"], **fit_params)
    
    return train_mod, test_mod, sample_weight

In [7]:
def objective(trial, dataset_name='bank_marketing', rng=42):
    train, test, hidden = split(dataset_name, random_state=rng)
    
    apply_pre = trial.suggest_categorical('pre', [0, 1, 2, 3, 4])
    apply_in = trial.suggest_categorical('in',[0, 1, 2, 3])
    
    #LFR and MetaFairClassifier dont work together
    if apply_pre == 3 and apply_in == 1:
        return 1e6
    
    name, proc = pre_mitigators[apply_pre]
    train_mod, test_mod, sample_weight = process_and_evaluate(name, proc, train, test)
    
    fit_params = {"sample_weight": sample_weight} if sample_weight is not None else {}
    
    name, proc = in_mitigators[apply_in]
    print(name)
    if name is not None:
        proc.estimator=LG(verbose=-1)
        proc.fit(train_mod, train['y'], train["group_a"], train["group_b"], **fit_params)
    else:
        proc = LG(verbose=-1)
        proc.fit(train_mod, train['y'], **fit_params)
    
    proxy = BinaryClassificationProxy(predict=proc.predict, predict_proba=proc.predict_proba)
    pred = proxy.predict(test_mod)
    
    
    #Create bias and performance metrics
    bias = classification_bias_metrics(test['group_a'], test['group_b'], pred, test['y'])
    bias_loss = abs(bias['Reference'] - bias['Value']).mean()
    
    #Create efficacy metrics, focusing on balanced acc and f1-score
    effic = classification_efficacy_metrics(pred, test['y'])
    effic = effic.iloc[[1,4]]
    effic_loss = abs(effic['Reference'] - effic['Value']).mean()
    display(bias)
    display(effic)

    # Feature Importance
    importance = compute_permutation_importance(proxy, X=train_mod, y=train['y'], n_repeats=1, n_jobs=1)
    top_importance = importance.top_alpha()
    top_feature_names = top_importance.feature_names

    # Conditional Feature Importance
    conditional_impotance = compute_conditional_permutation_importance(proxy, X=test_mod, y=test['y'], n_jobs=1)

    # Partial Dependence
    #mask = np.random.randint(0, len(train_mod), size=1000)
    X_sample = train_mod.sample(1000)
    partial_dependence = compute_partial_dependence(features=top_feature_names, proxy=proxy, X=X_sample)
    
    #Compute and display 2 diff XAI metrics
    feat_imp = classification_global_feature_importance_explainability_metrics(partial_dependence, importance, conditional_impotance)
    feat_imp_2 = classification_explainability_metrics(importance, partial_dependence, conditional_impotance)
    feat_imp_2.columns = feat_imp.columns
    
    final_feat = pd.concat([feat_imp, feat_imp_2], axis=0).drop_duplicates()
    feat_loss = abs(final_feat['Reference'] - final_feat['Value']).mean()
    display(final_feat)
    
    total = bias_loss + effic_loss + feat_loss
    
    loss = [bias_loss, effic_loss, feat_loss]
    
    print("Loss per metric:")
    print("Bias Loss:", bias_loss)
    print("Performance Loss:", effic_loss)
    print("Explainability Loss:", feat_loss)
    print("Total Loss:", total)
    
    return total

In [8]:
import optuna

studies = []
RANDOM_STATES = [42, 12312, 64759]
for dataset in ['bank_marketing', 'compas_two_year_recid', 'mw_small']:
    for random_state in RANDOM_STATES:
        search_space={
                 "pre":[0,1,2,3,4], 
                 "in":[0,1,2,3]
                 }#5*4 = 20 trials to exhaust
        study = optuna.create_study(direction='minimize',
                                   sampler=optuna.samplers.GridSampler(search_space)
                                   )
        func = lambda trial: objective(trial, dataset, random_state)

        study.optimize(func, n_trials=20)

        studies.append([dataset, random_state, study])

[I 2024-11-28 04:11:04,903] A new study created in memory with name: no-name-95957969-0d99-417d-9310-8052021e77e2
2024-11-28 04:11:05,509 - INFO - Unable to initialize backend 'cuda': 
2024-11-28 04:11:05,510 - INFO - Unable to initialize backend 'rocm': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
2024-11-28 04:11:05,512 - INFO - Unable to initialize backend 'tpu': INTERNAL: Failed to open libtpu.so: libtpu.so: cannot open shared object file: No such file or directory
2024-11-28 04:11:05,513 - WARNING - An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


LearningFairRepresentation
ExponentiatedGradientReduction


Value  Reference
Metric                                                 
Statistical Parity                 -0.006349          0
Disparate Impact                    0.763182          1
Four Fifths Rule                    0.763182          1
Cohen D                            -0.042408          0
2SD Rule                           -1.760419          0
Equality of Opportunity Difference -0.000944          0
False Positive Rate Difference     -0.003972          0
Average Odds Difference            -0.002458          0
Accuracy Difference                 0.032385          0

Value  Reference
Metric                                
Balanced Accuracy  0.700351          1
F1-Score           0.163776          1

Value Reference
Spread Divergence   0.43734         1
Fluctuation Ratio  0.002525         0
Rank Alignment     0.273897         1
Alpha Score        0.404255         0
XAI Ease Score     0.605263       1.0
Position Parity    0.138478       1.0
Rank Alignment     0.251596       1.0
Spread Ratio       0.850366       0.0

...

Value  Reference
Metric                                                  
Statistical Parity                   0.131535          0
Disparate Impact                     2.690754          1
Four Fifths Rule                     0.371643          1
Cohen D                              0.381207          0
2SD Rule                            18.344276          0
Equality of Opportunity Difference   0.072533          0
False Positive Rate Difference       0.135638          0
Average Odds Difference              0.104086          0
Accuracy Difference                 -0.165273          0

Value  Reference
Metric                                
Balanced Accuracy  0.505725          1
F1-Score           0.106549          1

Value Reference
Spread Divergence  0.661817         1
Fluctuation Ratio   0.24031         0
Rank Alignment     0.666667         1
Alpha Score            0.25         0
XAI Ease Score         0.25       1.0
Position Parity        0.75       1.0
Spread Ratio       0.442787       0.0

[I 2024-11-28 05:24:42,929] Trial 19 finished with value: 3.4719125978467327 and parameters: {'pre': 2, 'in': 2}. Best is trial 12 with value: 0.9515454168909379.


Loss per metric:
Bias Loss: 2.4059619921847415
Performance Loss: 0.6938629344116012
Explainability Loss: 0.37208767125038983
Total Loss: 3.4719125978467327


In [9]:
studies = np.array(studies)
dataset_dic={}
for dataset in studies[:, 0]:
    dataset_dic[dataset]=[]
    
for study in studies:
    trial_dic={}
    
    name, rng, details = study
    for c,i in enumerate(details.trials):
        trial_dic[c] = {'pre':i.params['pre'],
                        'in':i.params['in'],
                       'loss':i.values[0]}
        #print(i.values, i.params)
    trial_dic = pd.DataFrame(trial_dic).T
    trial_dic = trial_dic.sort_values(by=['pre','in'])
    
    dataset_dic[name].append(trial_dic)
    #display(trial_dic)

for dataset in list(dataset_dic.keys()):
    dataset_dic[dataset] = sum(dataset_dic[dataset])/len(dataset_dic[dataset])
    dataset_dic[dataset] = dataset_dic[dataset].reset_index(drop=True)
    dataset_dic[dataset].index.name = dataset
    display(dataset_dic[dataset])

pre   in            loss
bank_marketing                          
0               0.0  0.0        1.185255
1               0.0  1.0        1.167383
2               0.0  2.0        3.473580
3               0.0  3.0        1.175070
4               1.0  0.0        1.644330
5               1.0  1.0        1.213037
6               1.0  2.0        6.641251
7               1.0  3.0        1.260938
8               2.0  0.0        1.330722
9               2.0  1.0        1.088817
10              2.0  2.0        3.396630
11              2.0  3.0        1.246365
12              3.0  0.0        1.116346
13              3.0  1.0  1000000.000000
14              3.0  2.0        1.545157
15              3.0  3.0        1.294186
16              4.0  0.0        1.120132
17              4.0  1.0        0.992988
18              4.0  2.0        3.470483
19              4.0  3.0        1.185120

pre   in            loss
compas_two_year_recid                          
0                      0.0  0.0        1.282600
1                      0.0  1.0        1.153507
2                      0.0  2.0        1.706651
3                      0.0  3.0        1.242368
4                      1.0  0.0        1.631036
5                      1.0  1.0        1.036630
6                      1.0  2.0        1.179240
7                      1.0  3.0        1.128802
8                      2.0  0.0        1.233667
9                      2.0  1.0        1.078773
10                     2.0  2.0        1.578991
11                     2.0  3.0        1.167275
12                     3.0  0.0        1.518327
13                     3.0  1.0  1000000.000000
14                     3.0  2.0        1.659346
15                     3.0  3.0        1.395997
16                     4.0  0.0        1.260283
17                     4.0  1.0        1.164068
18                     4.0  2.0        1.697217
19                     4.0  3.0        1.229739

pre   in          loss
mw_small                        
0         0.0  0.0  2.009303e+00
1         0.0  1.0  1.310137e+00
2         0.0  2.0  4.308821e+00
3         0.0  3.0  1.750362e+00
4         1.0  0.0  2.812685e+00
5         1.0  1.0  2.030018e+00
6         1.0  2.0  1.890050e+02
7         1.0  3.0  1.659338e+00
8         2.0  0.0  1.594876e+00
9         2.0  1.0  1.861562e+00
10        2.0  2.0  6.329581e+00
11        2.0  3.0  1.634374e+00
12        3.0  0.0           inf
13        3.0  1.0  1.000000e+06
14        3.0  2.0  2.457408e+00
15        3.0  3.0  1.537672e+00
16        4.0  0.0  1.813314e+00
17        4.0  1.0  1.110064e+00
18        4.0  2.0  4.308170e+00
19        4.0  3.0  1.750233e+00

In [11]:
for dataset in list(dataset_dic.keys()):
    df = dataset_dic[dataset].copy()
    base_loss =  df.iloc[0]['loss']
    
    print(dataset)
    print("Baseline", base_loss)
    
    ans = df[df['loss']<base_loss]
    if len(ans)>0:
        print("There is a better pipeline than baseline, choosing the best based on combined loss")
        idx =  df['loss'].argmin()
        print("Best loss", df['loss'][idx])
        pre = df.iloc[idx]['pre'].astype(int)
        inn =  df.iloc[idx]['in'].astype(int)
        
        print(pre, inn)
        print("Best combo:",pre_mitigators[pre][0],"followed by", in_mitigators[inn][0])
    else:
        print("Current pipelines don't perform better than baseline")
    
    print()

bank_marketing
Baseline 1.185254652468404
There is a better pipeline than baseline, choosing the best based on combined loss
Best loss 0.9929876929754009
4 1
Best combo: Reweighing followed by MetaFairClassifier

compas_two_year_recid
Baseline 1.2825998242767198
There is a better pipeline than baseline, choosing the best based on combined loss
Best loss 1.036629529688371
1 1
Best combo: CorrelationRemover followed by MetaFairClassifier

mw_small
Baseline 2.009302654107616
There is a better pipeline than baseline, choosing the best based on combined loss
Best loss 1.1100643381099544
4 1
Best combo: Reweighing followed by MetaFairClassifier

